In [1]:
import pandas as pd

from utils import insert_df_to_db

In [2]:
tse_candidates = pd.read_csv('../data/2017-07-20-tse-candidates.xz', low_memory=False)
tse_candidates.head()

,year,phase,description,state,location,post,name,electoral_id,cpf,voter_id,result
0,2004,1,ELEICOES 2004,AC,ACRELANDIA,mayor,JOÃO DE ALMEIDA,6,18139124249,000356682496,not_elected
1,2004,1,ELEICOES 2004,AC,ACRELANDIA,mayor,PAULO CÉSAR FERREIRA DE ARAUJO,1,07857136204,000525772445,not_elected
2,2004,1,ELEICOES 2004,AC,ACRELANDIA,mayor,SEBASTIÃO BOCALOM RODRIGUES,27,17357152987,002053882488,elected
3,2004,1,ELEICOES 2004,AC,ACRELANDIA,vice_mayor,ERAIDES CAETANO DE SOUZA,7,40917860900,002817512496,NaN
4,2004,1,ELEICOES 2004,AC,ACRELANDIA,vice_mayor,RICARDO MODESTO DE OLIVEIRA,2,61109029268,002600402445,NaN


## Count elections result per candidate

In [3]:
dd = tse_candidates.groupby(['cpf', 'result'])['result'].count()

In [4]:
elections = dd.unstack()
elections.head()

result,alternate,elected,elected_by_party_quota,not_elected,rejected,replaced,runoff
cpf,,,,,,,
#NULO#,249.0,57.0,NaN,108.0,7.0,1.0,NaN
0,2.0,1.0,NaN,NaN,NaN,NaN,NaN
00000000000,4.0,NaN,NaN,4.0,1.0,NaN,NaN
00000012017,1.0,NaN,NaN,NaN,NaN,NaN,NaN
00000165000,1.0,NaN,NaN,NaN,NaN,NaN,NaN


## Get personal information of all candidates

In [5]:
candidates = tse_candidates.drop_duplicates(subset='cpf')[['name', 'cpf', 'voter_id', 'state']]
candidates.rename(columns={'name': 'civil_name'})
candidates.head()

,name,cpf,voter_id,state
0,JOÃO DE ALMEIDA,18139124249,000356682496,AC
1,PAULO CÉSAR FERREIRA DE ARAUJO,07857136204,000525772445,AC
2,SEBASTIÃO BOCALOM RODRIGUES,17357152987,002053882488,AC
3,ERAIDES CAETANO DE SOUZA,40917860900,002817512496,AC
4,RICARDO MODESTO DE OLIVEIRA,61109029268,002600402445,AC


In [6]:
candidates_elections = pd.merge(candidates, elections, on='cpf')
candidates_elections.head()

,name,cpf,voter_id,state,alternate,elected,elected_by_party_quota,not_elected,rejected,replaced,runoff
0,JOÃO DE ALMEIDA,18139124249,000356682496,AC,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1,PAULO CÉSAR FERREIRA DE ARAUJO,07857136204,000525772445,AC,NaN,NaN,NaN,1.0,NaN,NaN,NaN
2,SEBASTIÃO BOCALOM RODRIGUES,17357152987,002053882488,AC,NaN,1.0,NaN,5.0,NaN,NaN,1.0
3,RICARDO MODESTO DE OLIVEIRA,61109029268,002600402445,AC,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,VILSEU FERREIRA DA SILVA,27278913187,000411782470,AC,NaN,1.0,NaN,NaN,1.0,NaN,NaN


In [7]:
# Rename columsn of number of elections by status to elections_status
# Ex:  elected to elections_elected

for col in elections.columns.values:
    candidates_elections.rename(columns={col: 'elections_{}'.format(col)}, inplace=True)

In [8]:
candidates_elections.head(2)

,name,cpf,voter_id,state,elections_alternate,elections_elected,elections_elected_by_party_quota,elections_not_elected,elections_rejected,elections_replaced,elections_runoff
0,JOÃO DE ALMEIDA,18139124249,000356682496,AC,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1,PAULO CÉSAR FERREIRA DE ARAUJO,07857136204,000525772445,AC,NaN,NaN,NaN,1.0,NaN,NaN,NaN


## Get type of elections won

Gets how many times the candidate was elected or elected by party quota to each available post 

In [9]:
elected_candidates = tse_candidates[tse_candidates.result.isin(['elected', 'elected_by_party_quota'])]
num_election_won = elected_candidates.groupby(['cpf', 'post'])['result'].count()
num_election_won = num_election_won.unstack()
num_election_won.head()

post,city_councilman,district_deputy,federal_deputy,governor,mayor,president,senate_first_alternate,senate_second_alternate,senator,state_deputy,vice_governor,vice_mayor,vice_president
cpf,,,,,,,,,,,,,
#NULO#,54.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00000602590,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00000634107,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00000866571,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
candidates_elections = pd.merge(candidates_elections, num_election_won, on='cpf')

In [11]:
for col in num_election_won.columns.values:
    candidates_elections.rename(columns={col: 'times_elected_to_{}'.format(col)}, inplace=True)

In [12]:
candidates_elections.shape

(167024, 24)

In [13]:
candidates_elections.rename(columns={'name': 'civil_name'}, inplace=True)
candidates_elections.to_csv('../data/datalake/candidates_elections.csv', index=False)

In [14]:
df = candidates_elections[candidates_elections.times_elected_to_federal_deputy>0]
df.shape

(971, 24)

In [15]:
main_info = candidates_elections[[
    'civil_name', 
    'cpf', 
    'voter_id', 
    'state',
]]

In [16]:
insert_df_to_db(main_info.sample(30000))

{'created': 30000, 'updated': 0}

In [17]:
elections_info = candidates_elections[[
    'cpf',
    'elections_alternate',
    'elections_elected', 
    'elections_elected_by_party_quota',
    'elections_not_elected', 
    'elections_rejected', 
    'elections_replaced',
    'elections_runoff', 
    'times_elected_to_city_councilman',
    'times_elected_to_district_deputy', 
    'times_elected_to_federal_deputy',
    'times_elected_to_governor', 
    'times_elected_to_mayor',
    'times_elected_to_president', 
    'times_elected_to_senate_first_alternate',
    'times_elected_to_senate_second_alternate', 
    'times_elected_to_senator',
    'times_elected_to_state_deputy', 
    'times_elected_to_vice_governor',
    'times_elected_to_vice_mayor', 
    'times_elected_to_vice_president'
]]

# Add info to mongodb


In [18]:
insert_df_to_db(elections_info.fillna(0).sample(30000), 'elections_count')

{'created': 24507, 'updated': 5493}